In [11]:
import pandas as pd
import requests
import json

In [12]:
g_perth_sa2 = pd.read_csv("SA2_2016_AUST.csv")
g_perth_sa2.head()

,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099


In [13]:
g_perth_sa2 = g_perth_sa2.loc[g_perth_sa2["GCCSA_NAME_2016"] == "Greater Perth"]
g_perth_cols = ["SA2_MAINCODE_2016", "SA2_NAME_2016","GCCSA_NAME_2016"]
g_perth_sa2= g_perth_sa2[g_perth_cols].copy()
g_perth_sa2.dtypes
g_perth_sa2["SA2_MAINCODE_2016"] = g_perth_sa2["SA2_MAINCODE_2016"].apply(str)

In [14]:
#make a list of SA2 Maincode for ABS API
sa2_list = g_perth_sa2["SA2_MAINCODE_2016"].tolist()
edu_dic = {"SA2 Code" : [], "SA2 Name":[], "Education Level":[], "Age Group":[], "2006":[], "2011":[], "2016":[] }

In [15]:
# Rent 2006 2011 2016
rent_url = "http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T31_TS_SA/3.3.T75.5.SA2.504011046/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016"
bins_edu = ['1','2','3']
bins_age  = ['25','35','45','55','65','75']

for sa2 in sa2_list:
        for bins in bins_edu:
            for age in bins_age:
                 response = requests.get(f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T31_TS_SA/3.{bins}.T{age}.5.SA2.{sa2}/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016")
                 bacherlor_25 = response.json()
#                  print(bacherlor_25)\n",
                 edu_dic["SA2 Code"].append(bacherlor_25["structure"]["dimensions"]["observation"][5]["values"][0]["id"])
                 edu_dic["SA2 Name"].append(bacherlor_25["structure"]["dimensions"]["observation"][5]["values"][0]["name"])
                 edu_dic["Education Level"].append(bacherlor_25["structure"]["dimensions"]["observation"][1]["values"][0]["name"])
                 edu_dic["Age Group"].append(bacherlor_25["structure"]["dimensions"]["observation"][2]["values"][0]["name"])
                 edu_dic["2006"].append(bacherlor_25["dataSets"][0]["observations"]["0:0:0:0:0:0:0"][0])
                 edu_dic["2011"].append(bacherlor_25["dataSets"][0]["observations"]["0:0:0:0:0:0:1"][0])
                 edu_dic["2016"].append(bacherlor_25["dataSets"][0]["observations"]["0:0:0:0:0:0:2"][0])
    
    

In [16]:
edu_df = pd.DataFrame(edu_dic)
print(edu_df)
total_dic = {"SA2 Code" : [], "SA2 Name":[], "Education Level":[], "Age Group":[], "2006":[], "2011":[], "2016":[] }

       SA2 Code              SA2 Name            Education Level Age Group  \
0     502011021  Dawesville - Bouvard  Postgraduate Degree Level   25 - 34   
1     502011021  Dawesville - Bouvard  Postgraduate Degree Level   35 - 44   
2     502011021  Dawesville - Bouvard  Postgraduate Degree Level   45 - 54   
3     502011021  Dawesville - Bouvard  Postgraduate Degree Level   55 - 64   
4     502011021  Dawesville - Bouvard  Postgraduate Degree Level   65 - 74   
...         ...                   ...                        ...       ...   
3109  507051193               Warnbro      Bachelor Degree Level   35 - 44   
3110  507051193               Warnbro      Bachelor Degree Level   45 - 54   
3111  507051193               Warnbro      Bachelor Degree Level   55 - 64   
3112  507051193               Warnbro      Bachelor Degree Level   65 - 74   
3113  507051193               Warnbro      Bachelor Degree Level   75 - 84   

       2006   2011   2016  
0       0.0    7.0   16.0  
1      

In [17]:
 edu_df.to_csv('Education Level 2006-2016.csv')

In [18]:
# # Rent 2006 2011 2016
# rent_url = "http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T31_TS_SA/3.TOT.TOT.5.SA2.507011152/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016"
# # bins_edu = ['1','2','3']
# # bins_age  = ['25','35','45','55','65','75']

# for sa2 in sa2_list:
# #         for bins in bins_edu:
# #             for age in bins_age:
#                  response = requests.get(f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T31_TS_SA/3.TOT.TOT.5.SA2.{sa2}/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016")
#                  total = response.json()
#                  total_dic["SA2 Code"].append(total["structure"]["dimensions"]["observation"][5]["values"][0]["id"])
#                  total_dic["SA2 Name"].append(total["structure"]["dimensions"]["observation"][5]["values"][0]["name"])
#                  total_dic["Education Level"].append(total["structure"]["dimensions"]["observation"][1]["values"][0]["name"])
#                  total_dic["Age Group"].append(total["structure"]["dimensions"]["observation"][2]["values"][0]["name"])
#                  total_dic["2006"].append(total["dataSets"][0]["observations"]["0:0:0:0:0:0:0"][0])
#                  total_dic["2011"].append(total["dataSets"][0]["observations"]["0:0:0:0:0:0:1"][0])
#                  total_dic["2016"].append(total["dataSets"][0]["observations"]["0:0:0:0:0:0:2"][0])
#                  print(total_dic)
    
    

{'SA2 Code': ['502011021'], 'SA2 Name': ['Dawesville - Bouvard'], 'Education Level': ['Total'], 'Age Group': ['Total'], '2006': [1768.0], '2011': [2218.0], '2016': [3309.0]}
{'SA2 Code': ['502011021', '502011022'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup'], 'Education Level': ['Total', 'Total'], 'Age Group': ['Total', 'Total'], '2006': [1768.0, 2465.0], '2011': [2218.0, 3206.0], '2016': [3309.0, 4187.0]}
{'SA2 Code': ['502011021', '502011022', '502011023'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields'], 'Education Level': ['Total', 'Total', 'Total'], 'Age Group': ['Total', 'Total', 'Total'], '2006': [1768.0, 2465.0, 3243.0], '2011': [2218.0, 3206.0, 3599.0], '2016': [3309.0, 4187.0, 3985.0]}
{'SA2 Code': ['502011021', '502011022', '502011023', '502011024'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields', 'Halls Head - Erskine'], 'Education Level': ['Total', 'Total', 'Total', 'Total'], 'Age Group': ['Total', 'Total', 'T

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields', 'Halls Head - Erskine', 'Mandurah', 'Mandurah - East', 'Mandurah - North', 'Mandurah - South', 'Pinjarra', 'City Beach', 'Claremont (WA)', 'Cottesloe', 'Floreat', 'Mosman Park - Peppermint Grove', 'Nedlands - Dalkeith - Crawley'], 'Education Level': ['Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total'], 'Age Group': ['Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total'], '2006': [1768.0, 2465.0, 3243.0, 5706.0, 3171.0, 1973.0, 2751.0, 3154.0, 1817.0, 3208.0, 4051.0, 4200.0, 3521.0, 5231.0, 9695.0], '2011': [2218.0, 3206.0, 3599.0, 7418.0, 3733.0, 2

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields', 'Halls Head - Erskine', 'Mandurah', 'Mandurah - East', 'Mandurah - North', 'Mandurah - South', 'Pinjarra', 'City Beach', 'Claremont (WA)', 'Cottesloe', 'Floreat', 'Mosman Park - Peppermint Grove', 'Nedlands - Dalkeith - Crawley', 'Swanbourne - Mount Claremont', 'Kings Park (WA)', 'Mount Hawthorn - Leederville', 'Mount Lawley - Inglewood', 'North Perth', 'Perth City', 'Subiaco - Shenton Park'], 'Education Level': ['Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total', 'Total'], 'Age Group

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields', 'Halls Head - Erskine', 'Mandurah', 'Mandurah - East', 'Mandurah - North', 'Mandurah - South', 'Pinjarra', 'City Beach', 'Claremont (WA)', 'Cottesloe', 'Floreat', 'Mosman Park - Peppermint Grove', 'Nedlands - Dalkeith - Crawley', 'Swanbourne - Mount Claremont', 'Kings Park (WA)', 'Mount Hawthorn - Leederville', 'Mount Lawley - Inglewood', 'North Perth', 'Perth City', 'Subiaco - Shenton Park', 'Wembley - West Leederville - Glendalough', 'Bassendean - Eden Hill - Ashfield', 'Bayswater - Embleton - Bedford', 'Maylands', 'Morley', 'Noranda'], 'Ed

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052'], 'SA2 Name': ['Dawesville - Bouvard', 'Falcon - Wannanup', 'Greenfields', 'Halls Head - Erskine', 'Mandurah', 'Mandurah - East', 'Mandurah - North', 'Mandurah - South', 'Pinjarra', 'City Beach', 'Claremont (WA)', 'Cottesloe', 'Floreat', 'Mosman Park - Peppermint Grove', 'Nedlands - Dalkeith - Crawley', 'Swanbourne - Mount Claremont', 'Kings Park (WA)', 'Mount Hawthorn - Leederville', 'Mount Lawley - Inglewood', 'North Perth', 'Perth City', 'Subiaco - Shenton Park', 'Wembley - West Leederville - Glendalough', 'Bassendean - Eden Hill - Ashfield', 'Bayswater - Emblet

...

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052', '504021053', '504021054', '504031055', '504031056', '504031057', '504031058', '504031059', '504031060', '504031061', '504031062', '504031063', '504031064', '504031065', '504031066', '504031067', '504031068', '504031069', '505011070', '505011071', '505011072', '505011073', '505011074', '505011075', '505011076', '505011077', '505011078', '505011079', '505011080', '505011081', '505011082', '505011083', '505021084', '505021085', '505021086', '505021087', '505021088', '505021089', '505021090', '505021091', '505021092', '505021093', '505021094', '505021095', '505021096'

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052', '504021053', '504021054', '504031055', '504031056', '504031057', '504031058', '504031059', '504031060', '504031061', '504031062', '504031063', '504031064', '504031065', '504031066', '504031067', '504031068', '504031069', '505011070', '505011071', '505011072', '505011073', '505011074', '505011075', '505011076', '505011077', '505011078', '505011079', '505011080', '505011081', '505011082', '505011083', '505021084', '505021085', '505021086', '505021087', '505021088', '505021089', '505021090', '505021091', '505021092', '505021093', '505021094', '505021095', '505021096'

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052', '504021053', '504021054', '504031055', '504031056', '504031057', '504031058', '504031059', '504031060', '504031061', '504031062', '504031063', '504031064', '504031065', '504031066', '504031067', '504031068', '504031069', '505011070', '505011071', '505011072', '505011073', '505011074', '505011075', '505011076', '505011077', '505011078', '505011079', '505011080', '505011081', '505011082', '505011083', '505021084', '505021085', '505021086', '505021087', '505021088', '505021089', '505021090', '505021091', '505021092', '505021093', '505021094', '505021095', '505021096'

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052', '504021053', '504021054', '504031055', '504031056', '504031057', '504031058', '504031059', '504031060', '504031061', '504031062', '504031063', '504031064', '504031065', '504031066', '504031067', '504031068', '504031069', '505011070', '505011071', '505011072', '505011073', '505011074', '505011075', '505011076', '505011077', '505011078', '505011079', '505011080', '505011081', '505011082', '505011083', '505021084', '505021085', '505021086', '505021087', '505021088', '505021089', '505021090', '505021091', '505021092', '505021093', '505021094', '505021095', '505021096'

{'SA2 Code': ['502011021', '502011022', '502011023', '502011024', '502011025', '502011026', '502011027', '502011028', '502011029', '503011030', '503011031', '503011032', '503011033', '503011034', '503011035', '503011036', '503021037', '503021038', '503021039', '503021040', '503021041', '503021042', '503021043', '504011044', '504011045', '504011046', '504011047', '504011048', '504021049', '504021050', '504021051', '504021052', '504021053', '504021054', '504031055', '504031056', '504031057', '504031058', '504031059', '504031060', '504031061', '504031062', '504031063', '504031064', '504031065', '504031066', '504031067', '504031068', '504031069', '505011070', '505011071', '505011072', '505011073', '505011074', '505011075', '505011076', '505011077', '505011078', '505011079', '505011080', '505011081', '505011082', '505011083', '505021084', '505021085', '505021086', '505021087', '505021088', '505021089', '505021090', '505021091', '505021092', '505021093', '505021094', '505021095', '505021096'

In [19]:
# total_df = pd.DataFrame(total_dic)
# print(total_df)

In [20]:
# total_df.to_csv('total_edu.csv')